<a href="https://colab.research.google.com/github/ehpub/kfq_ML_2023/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%ED%8F%89%EC%A0%90_%EA%B0%90%EC%84%B1_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

강의 교재 : https://wikibook.co.kr/pymlrev2/

In [1]:
import pandas as pd

In [ ]:
!pip install konlpy

In [4]:
from konlpy.tag import Twitter

In [5]:
path1 = '/content/drive/MyDrive/mldata/ratings_train.txt'
path2 = '/content/drive/MyDrive/mldata/ratings_test.txt'

In [6]:
train_df = pd.read_csv(path1,sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [8]:
test_df = pd.read_csv(path2,sep='\t')

In [9]:
train_df = train_df.drop('id',axis=1)
test_df = test_df.drop('id',axis=1)

In [24]:
fil = train_df['document'].notnull()
train_df = train_df[fil]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  149995 non-null  object
 1   label     149995 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.4+ MB


In [25]:
fil = test_df['document'].notnull()
test_df = test_df[fil]
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  49997 non-null  object
 1   label     49997 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [26]:
import re

In [27]:
def remove_number(text):
  return re.sub(r"\d"," ",text)

In [29]:
test_df['document'] = test_df['document'].apply(remove_number)
train_df['document'] = train_df['document'].apply(remove_number)

In [30]:
from konlpy.tag import Twitter

In [31]:
def dw_tokenizer(text):
  tw = Twitter()
  return tw.morphs(text)

In [32]:
dw_tokenizer('나는 집에 간다. 그리고 우리는 산에 간다.')

/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['나', '는', '집', '에', '간다', '.', '그리고', '우리', '는', '산', '에', '간다', '.']

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
trl = train_df['document'].values.tolist()
tel = test_df['document'].values.tolist()
tol = trl+tel
tdf = pd.DataFrame()
tdf['document'] = tol

In [38]:
tv = TfidfVectorizer(tokenizer=dw_tokenizer,
                ngram_range=(1,2),
                min_df=0.003,
                max_df=0.9)
tv.fit(tdf['document'])
train_data = tv.transform(train_df['document'])
test_data = tv.transform(test_df['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [41]:
model = LogisticRegression(C=3.5)
model.fit(train_data,train_df['label'])
pred = model.predict(test_data)
accuracy_score(test_df['label'],pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.768806128367702